## Conociendo un poco nuestro DATASET

#### Las limpiezas que se deben realizar son las siguientes:

1- `belongs_to_collection`, `production_companies`, `genres`, `spoken_languages`: los datos estan anidados
    Se debe buscar la forma de desanidarlos, o bien, buscar la forma de que los datos se puedan consultar sin desanidarlos.

2- `revenue`, `budget`: poseen datos nulos.
    Se deben rellenar estos datos con *0*.

3- `release_date`: posee datos nulos.
    Se deben eliminar estos datos.

4- `Campos con fecha`: no poseen el formato correcto.
    Se debe transformar cada uno de estos campos al formato *AAAA-mm-dd*.
    Se debe crear una nueva columna `release_year` donde se extraerá el año de la fecha de estreno.

5- `revenue`, `budget`: a partir de estos campos se debe crear una nueva columna `return`, retorno de inversión.
    Esta columna sera el resultado de `revenue`/`budget` y cuando no hay datos disponibles debera retornar *0*.
    
6- `video`, `imdb_id`, `adult`, `original_title`, `poster_path`, `homepage`: no utilizaremos estos campos.
    Se deberan eliminar.

In [79]:
# importamos las librerias necesarias para limpieza y transformacion:
import pandas as pd
import numpy as np

### Movies

### Pequeño análisis exploratorio

In [80]:
# creamos nuestro dataframe
mv = pd.read_csv('movies_dataset.csv', delimiter=',')
mv.tail(4)

C:\Users\Martin\AppData\Local\Temp\ipykernel_11404\780545667.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  mv = pd.read_csv('movies_dataset.csv', delimiter=',')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [81]:
mv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [82]:
# vemos el tamaño de nuestro dataframe
mv.shape

(45466, 24)

In [83]:
# vemos sus columnas
mv.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [84]:
mv.id.duplicated().value_counts() # tenemos duplicados, pero la consigna no nos dice que debemos borrarlos.

False    45436
True        30
Name: id, dtype: int64

---------------------
### Se podria comenzar por el item 6 para tener un numero de columnas mas reducido y un mejor uso de los recursos a la hora de realizar las trasnformaciones. No es lo mismo transformar un df con 24 columnas que un df con 18 columnas.
------

### 6- `video`, `imdb_id`, `adult`, `original_title`, `poster_path`, `homepage`: no utilizaremos estos campos.
    Se deberan eliminar.

In [85]:
mv.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

In [86]:
# verificamos que los campos se hayan borrado
mv.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

-----
### Con un razonamiento parecido al anterior, seria recomendable seguir por el punto 3. En este punto, se deben eliminar registros en los que 'release_date' posea datos nulos. Con la eliminacion de estos registros estariamos optimizando la utilizacion de los recursos de procesamiento. No es lo mismo un df de 44000 registros que un df de 40000.
-----

### 3- `release_date`: posee datos nulos.
    Se deben eliminar estos datos.

In [87]:
# inicialmente nuestro dataframe mv posee 45466 registros
mv.shape

(45466, 18)

In [88]:
mv.dropna(subset=['release_date'], inplace=True) # con el parametro subset defino en que columna voy a buscar los valores NaN para dropearlos

In [91]:
# una vez dropeados estos registros tendremos:
print(mv.shape)
print(f'Se han eliminado {45466-45379} registros')

(45379, 18)
Se han eliminado 87 registros


### 1- `belongs_to_collection`, `production_companies`, `genres`, `spoken_languages`: los datos estan anidados
    Se debe buscar la forma de desanidarlos, o bien, buscar la forma de que los datos se puedan consultar sin desanidarlos.

#### **belong_to_collection**

In [ ]:
# hacemos una copia de nuestra columna original para poder trabajarla
btc_copy = mv.belongs_to_collection # no es Bitcoin, es belong_to_collection

In [ ]:
# notemos que en nuestra columna 'belong_to_collection' podemos encontrar dos tipos de variables:

print(type(btc_copy[4]))
# donde tenemos los datos anidados, el tipo de dato sera string

print(type(btc_copy[3]))
# donde tenemos NaN, el tipo de dato sera float

Con esto se podria hacer una primer transformacion. La idea principal sería, que todos nuestros datos en 'belong_to_collection' sean diccionarios los cuales se pueda acceder facilmente a ellos.

------

##### **DEBO OPTIMIZAR ESTO!!!**

In [ ]:
### OPCION A: tarda mas de 20 segundos en ejecutarse


#   import ast # este modulo nos servira para transformar de string a diccionario
#   for i, registro in enumerate(btc_copy): # para cada registro en mi columna belng_to_collection
#       if (type(registro) == str):
#           btc_copy[i] = ast.literal_eval(btc_copy[i]) # transformamos str -> dict mediante funcion literal_eval
#       else: # sino, transformamos float -> dict
#           btc_copy[i] = {
#               'id':'Sin datos',
#               'name':'Sin datos',
#               'poster_path': 'Sin datos',
#               'backdrop_path': 'Sin datos'
#           }


### Resuelve el problema, pero es una locura que tarde 17 segundos

In [ ]:
### OPCION B: es 200 veces mas rapido que la OPCION A. Esto es por la funcion apply que usamos de Pandas, es mucho mas rapida
### que un bucle for tradicional
import ast

def transformar_registro_btc(registro):
    if isinstance(registro, str):
        nuevo_registro = ast.literal_eval(registro)
        nuevo_registro['id_belong_to_collection'] = nuevo_registro['id']
        del nuevo_registro['id']
        return nuevo_registro
    else:
        return {
            'id': 'Sin datos',
            'name': 'Sin datos',
            'poster_path': 'Sin datos',
            'backdrop_path': 'Sin datos'
        }

# Aplicar la función personalizada a la columna 'btc_copy'
btc_copy = btc_copy.apply(transformar_registro_btc)

# Si es necesario, convierte la columna 'btc_copy' en un DataFrame
# btc_copy = pd.DataFrame(btc_copy)

btc_copy


------

In [ ]:
print(btc_copy[4],type(btc_copy[4]))
print(btc_copy[3],type(btc_copy[3]))
# Efectivamente, logramos transformar todos los registros de esta columna a diccionarios

Ahora la pregunta sería, como podria transformar este array de diccionarios, en un Dataframe de Pandas en el que cada una de las keys de los diccionarios sea una columna??

In [ ]:
belong_to_collection_df = pd.json_normalize(btc_copy) # funcion para expandir diccionarios en columnas
# tambien me lo transforma en Dataframe

### Seria conveniente usar esta funcion para el momento en el que tenga que consultar los registros de btc_copy

Con esto, logre crear un Dataframe en el que pueda acceder a los datos de la columna 'belong_to_collection' fácilmente

In [ ]:
belong_to_collection_df[belong_to_collection_df['name']=='Toy Story Collection']

#### **production_companies**